In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

from importlib.util import find_spec
if find_spec("qml_hep_lhc") is None:
    import sys
    sys.path.append('..')

In [2]:
from qml_hep_lhc.data import ElectronPhoton, MNIST, QuarkGluon
from qml_hep_lhc.data.utils import tf_ds_to_numpy
import argparse
import wandb

import pennylane as qml
import jax.numpy as jnp
import jax
import optax
from jax.nn.initializers import he_uniform
from jax import grad, jit, vmap
from jax import random
import tensorflow_datasets as tfds
from tqdm import tqdm
import numpy as np
import tensorflow as tf

# Added to silence some warnings.
# from jax.config import config
# config.update("jax_enable_x64", True)

import matplotlib.pyplot as plt
import time

2022-09-04 21:29:18.454994: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-09-04 21:29:18.455097: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
jax.devices()

[CpuDevice(id=0)]

In [97]:
args = argparse.Namespace()

# Data
# args.center_crop = 0.2
# args.resize = [8,8]
args.standardize = 1
args.power_transform = 1
# args.binary_data = [3,6]
args.percent_samples = 0.01
# args.processed = 1
args.dataset_type = '1'
args.labels_to_categorical = 1
args.batch_size = 2
args.validation_split = 0.1

# Base Model
args.wandb = False
args.epochs = 50
args.learning_rate = 0.001

# Quantum CNN Parameters
args.n_layers = 1
args.n_qubits = 1
args.template = 'NQubitPQC'
args.initializer = 'he_uniform'

args.kernel_size = (3,3)
args.strides = (2,2)
args.padding = "VALID"

args.clayer_sizes = [8, 2]

In [98]:
if args.wandb:
     wandb.init(project='qml-hep-lhc', config = vars(args))

In [99]:
data = QuarkGluon(args)
data.prepare_data()
data.setup()
print(data)

Performing power transform...


/home/gopald/Documents/qml-hep-lhc/qenv/lib/python3.7/site-packages/sklearn/preprocessing/_data.py:3237: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


Standardizing data...
Converting labels to categorical...
Converting labels to categorical...

Dataset :Quark Gluon 1
╒════════╤═════════════════╤════════════════╤═════════════════╤═════════════╕
│ Data   │ Train size      │ Val size       │ Test size       │ Dims        │
╞════════╪═════════════════╪════════════════╪═════════════════╪═════════════╡
│ X      │ (81, 40, 40, 1) │ (9, 40, 40, 1) │ (10, 40, 40, 1) │ (40, 40, 1) │
├────────┼─────────────────┼────────────────┼─────────────────┼─────────────┤
│ y      │ (81, 2)         │ (9, 2)         │ (10, 2)         │ (2,)        │
╘════════╧═════════════════╧════════════════╧═════════════════╧═════════════╛

╒══════════════╤═══════╤════════╤════════╤═══════╤══════════════════════════╕
│ Type         │   Min │    Max │   Mean │   Std │ Samples for each class   │
╞══════════════╪═══════╪════════╪════════╪═══════╪══════════════════════════╡
│ Train Images │  -0.7 │   9.43 │  -0    │  0.99 │ [41, 40]                 │
├──────────────┼───────

## Hyperparameters

In [100]:
input_dims = data.config()['input_dims']
input_dims

(40, 40, 1)

In [101]:
def get_out_shape(in_shape, k, s, padding):
    in_shape = (1,) + in_shape
    a = np.random.uniform(size = (in_shape))
    dn = jax.lax.conv_dimension_numbers(a.shape, (1,1,k[0],k[1]), ('NHWC', 'IOHW', 'NHWC'))
    out = jax.lax.conv_general_dilated_patches(lhs = a,
                                           filter_shape= k,
                                           window_strides=s,
                                           padding=padding,
                                           dimension_numbers=dn 
                                    )
    return out.shape

In [102]:
initializer = he_uniform()

# Get qlayer sizes
def get_qlayer_sizes(template, n_l, n_q, k_size):
    if template == 'NQubitPQCSparse':
        return {
            'w': (n_l, n_q,3,np.prod(k_size)),
            'b': (n_l,n_q,3,1)
        }
    elif template == 'LightPQC':
        return {
            'w': (n_l,n_q,np.prod(k_size)),
            'b': (n_l,n_q,1)
        }
    elif template == 'NQubitPQC':
        assert np.prod(k_size)%3 == 0
        return {
            'w': (n_l,n_q,np.prod(k_size)),
            'b': (n_l,n_q,np.prod(k_size))
        }
    elif template == 'Qernel':
        assert n_q == 3
        assert np.prod(k_size)%3 == 0
        
        return {
            'w': (n_l, n_q, 3),
            'b': (n_l, n_q, 1),
        }

def random_qlayer_params(size, key, scale=1e-1):
    return initializer(key, size)
    return scale * random.normal(key, size)

def init_qnetwork_params(sizes, key):
    keys = random.split(key, len(sizes))
    return [[random_qlayer_params(size, key) for size, key in zip(sizes.values(), keys)]]
 

# A helper function to randomly initialize weights and biases
# for a dense neural network layer
def random_clayer_params(m, n, key, scale=1e-1):
    w_key, b_key = random.split(key)
    return initializer(w_key, (n,m)), random.normal(b_key, (n,))
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

# Initialize all layers for a fully-connected neural network with sizes "sizes"
def init_network_params(sizes, key):
    keys = random.split(key, len(sizes))
    return [random_clayer_params(m, n, k) for m, n, k in zip(sizes[:-1], sizes[1:], keys)]

kernel_size = args.kernel_size
strides = args.strides
padding = args.padding
clayer_sizes = args.clayer_sizes

template = args.template
n_layers = args.n_layers
n_qubits = args.n_qubits


conv_out_shape = get_out_shape(input_dims, kernel_size, strides, padding)
num_pixels = np.prod(conv_out_shape[:-1])*n_qubits
qlayer_sizes = get_qlayer_sizes(template, n_layers, n_qubits, kernel_size)
clayer_sizes = [num_pixels] + clayer_sizes

params = []
params += init_qnetwork_params(qlayer_sizes, random.PRNGKey(0))
# params += init_qnetwork_params(qlayer_sizes, random.PRNGKey(1))
params += init_network_params(clayer_sizes, random.PRNGKey(2))

/home/gopald/Documents/qml-hep-lhc/qenv/lib/python3.7/site-packages/jax/_src/lax/other.py:97: UserWarning: Explicitly requested dtype float64 requested in eye is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  rhs = jnp.eye(spatial_size, dtype=lhs.dtype).reshape(filter_shape * 2)


In [103]:
for i in params:
    for j in i:
        print(j.shape, end = ' ')
    print()

(1, 1, 9) (1, 1, 9) 
(8, 361) (8,) 
(2, 8) (2,) 


## QLayers

In [104]:
dev = qml.device('default.qubit.jax', wires=n_qubits)
qubits =list(range(n_qubits))

@jax.jit
@qml.qnode(dev, interface='jax')
def NQubitPQCSparse(inputs, w, b):
    z = jnp.dot(w, jnp.transpose(inputs))+ b

    for q in qubits:
        qml.Hadamard(wires=q)
    
    for l in range(n_layers):
        for q in qubits:
            qml.Rot(z[l,q,0], z[l,q,1], z[l,q,2], wires= q)
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))
   
#     return qml.expval(qml.PauliZ(qubits[-1]))
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [105]:
dev = qml.device('default.qubit.jax', wires=n_qubits)
qubits =list(range(n_qubits))

@jax.jit
@qml.qnode(dev, interface='jax')
def NQubitPQC(inputs, w, b):
    steps = inputs.shape[-1]//3
    for q in qubits:
        qml.Hadamard(wires=q)
    
    for l in range(n_layers):
        for q in qubits:
            for i in range(steps):
                z = jnp.transpose(jnp.multiply(inputs[:,3*i:3*i+3],w[l,q,3*i:3*i+3]) + b[l,q,3*i:3*i+3])
                qml.RZ(z[0], wires=q)
                qml.RY(z[1], wires=q)
                qml.RZ(z[2], wires=q)
                
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))

#     return qml.expval(qml.PauliZ(qubits[-1]))
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [106]:
dev = qml.device('default.qubit.jax', wires=n_qubits)
qubits =list(range(n_qubits))

@jax.jit
@qml.qnode(dev, interface='jax')
def LightPQC(inputs, w, b):
    steps = inputs.shape[-1]//3
    for q in qubits:
        qml.Hadamard(wires=q)
    
    for l in range(n_layers):
        for q in qubits:
            for i in range(steps):
                z = jnp.transpose(jnp.multiply(inputs[:,3*i:3*i+3],w[l,q,3*i:3*i+3]))
                qml.RZ(z[0], wires=q)
                qml.RY(z[1], wires=q)
                qml.RZ(z[2], wires=q)
            qml.RX(b[l,q,0], wires = q)
        if (l & 1):
            for q0, q1 in zip(qubits[1::2], qubits[2::2] + [qubits[0]]):
                qml.CZ((q0,q1))
        else:
            for q0, q1 in zip(qubits[0::2], qubits[1::2]):
                qml.CZ((q0,q1))
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [107]:
dev = qml.device('default.qubit.jax', wires=n_qubits)
qubits =list(range(n_qubits))

@jax.jit
@qml.qnode(dev, interface='jax')
def Qernel(inputs, w, b):
    inputs = jnp.transpose(inputs)
    batch_dim = inputs.shape[-1]

    for q in qubits:
        qml.Hadamard(wires=q)
    
    for l in range(n_layers):
        for q in qubits:
            z = jnp.multiply(inputs[3*q:3*q+3, :], jnp.transpose(jnp.tile(w[l,q], (batch_dim,1))))
            qml.Rot(z[0], z[1], z[2], wires= q)
            qml.RX(b[l,q,0], wires = q)
    return [qml.expval(qml.PauliZ(q)) for q in qubits]

In [108]:
def get_node(template):
    if template == 'NQubitPQC':
        return NQubitPQC
    elif template == 'LightPQC':
        return LightPQC
    elif template == 'NQubitPQCSparse':
        return NQubitPQCSparse
    elif template == 'Qernel':
        return Qernel

In [109]:
def qconv(x, qweights):
    x = jnp.expand_dims(x,axis=0)
    dn = jax.lax.conv_dimension_numbers(x.shape, 
                                        (1,1,kernel_size[0],kernel_size[1]), 
                                        ('NHWC', 'IOHW', 'NHWC'))
    x = jax.lax.conv_general_dilated_patches(lhs = x,
                                               filter_shape= kernel_size,
                                               window_strides=strides,
                                               padding=padding,
                                               dimension_numbers=dn 
                                              )
    iters = x.shape[1:3]
    x = jnp.reshape(x, (-1, np.prod(kernel_size)))
    
    x = get_node(template)(x, *qweights)
    
    x = jnp.reshape(x, iters + (n_qubits,))
    return x

In [110]:
dev = qml.device("default.qubit", wires=n_qubits)
qnode = qml.QNode(get_node(template), dev)

inputs = np.random.uniform(size = (10,np.prod(kernel_size)))
weights = params[0]
drawer = qml.draw(qnode, expansion_strategy="device")
print(drawer(inputs,*weights))

0: ──H──RZ(M0)──RY(M1)──RZ(M2)──RZ(M3)──RY(M4)──RZ(M5)──RZ(M6)──RY(M7)──RZ(M8)─┤  <Z>


## Auto-Batching Predictions

In [111]:
from jax.scipy.special import logsumexp

def relu(x):
    return jnp.maximum(0, x)

def forward(params, image):
  # per-example predictions
    activations = qconv(image, params[0])
#     activations += image
#     activations = relu(activations)
    activations = jnp.reshape(activations, (-1))
    for w, b in params[1:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b
    return logits - logsumexp(logits)

In [112]:
def forwardx(params, image):
  # per-example predictions
    activations = qconv(image, params[0])
    activations = relu(activations)
    activations = qconv(activations, params[1])
    activations += image
    activations = relu(activations)
    
    activations = jnp.reshape(activations, (-1))
    for w, b in params[2:-1]:
        outputs = jnp.dot(w, activations) + b
        activations = relu(outputs)
    final_w, final_b = params[-1]
    logits = jnp.dot(final_w, activations) + final_b
    return logits - logsumexp(logits)

In [113]:
# This works on single examples
random_flattened_image = random.normal(random.PRNGKey(1), input_dims)
random_flattened_image = jnp.floor(random_flattened_image*10)
preds = forward(params,  random_flattened_image)
print(preds)

[-48.50864   0.     ]


In [114]:
# Doesn't work with a batch
random_flattened_images = random.normal(random.PRNGKey(1), (2,)+ input_dims)
random_flattened_images = jnp.floor(random_flattened_images*10)
# try:
#     preds = predict(params, random_flattened_images)
# except TypeError:
#     print('Invalid shapes!')

In [115]:
# Let's upgrade it to handle batches using `vmap`

# Make a batched version of the `predict` function
batched_forward = vmap(forward, in_axes=(None,0))

# `batched_predict` has the same call signature as `predict`
batched_preds = batched_forward(params, random_flattened_images)
print(batched_preds)

[[-1.4724731e-03 -6.5211868e+00]
 [-9.1552734e-05 -9.3024044e+00]]


## Utility and loss functions

In [116]:
from sklearn.metrics import roc_auc_score

def accuracy(y_true, y_pred):
    target_class = jnp.argmax(y_true, axis=1)
    predicted_class = jnp.argmax(y_pred, axis=1)
    return jnp.sum(predicted_class == target_class)
 

def loss_fn(params, images, targets):
    preds = batched_forward(params, images)
    loss_value = -jnp.mean(preds * targets)
    return loss_value, preds

@jit
def update(opt_state, params, x, y):
    _ , grads = jax.value_and_grad(loss_fn, has_aux=True)(params, x, y)
    
    updates, opt_state = optimizer.update(grads, opt_state)
    params = optax.apply_updates(params, updates)
    
    return params, opt_state

@jit
def step(params,x,y):
    loss_value, preds = loss_fn(params, x, y)
    acc = accuracy(y, preds)
    return loss_value, acc

def evaluate(params, ds):
    losses = []
    accs = []
    with tqdm(tfds.as_numpy(ds), unit="batch") as tepoch:
        for x, y in tepoch:
            tepoch.set_description("Validation")
            loss_value, acc = step(params, x, y)
            losses.append(loss_value)
            accs.append(acc)
       
    return jnp.mean(np.array(losses)), jnp.mean(np.array(accs))/args.batch_size

def predict(params, ds):
    preds = []
    y_true = []
    with tqdm(tfds.as_numpy(ds), unit="batch") as tepoch:
        for x, y in tepoch:
            preds += list(batched_forward(params, x))
            y_true += list(y)
    
    return np.array(preds), np.array(y_true)

## Training loop

In [117]:
lr = 1e-3

In [118]:
schedule_fn = optax.linear_schedule(transition_steps=150,
                                    init_value=0.2,
                                    end_value=1e-7,
                                    )
# Defining an optimizer in Jax 
# optimizer = optax.adam(learning_rate=schedule_fn)

print(lr)
# optimizer = optax.adam(learning_rate=args.learning_rate)
optimizer = optax.adam(learning_rate=lr)
opt_state = optimizer.init(params)
lr = (lr*np.sqrt(0.1))

0.001


In [119]:
import time

# epochs = args.epochs
epochs = 100

epoch_times = []
for epoch in range(epochs):
    start_time = time.time()

    with tqdm(tfds.as_numpy(data.train_ds), unit="batch") as tepoch:
        for x, y in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            params, opt_state = update(opt_state, params, x, y)
        
    epoch_time = time.time() - start_time
    epoch_times.append(epoch_time)
    
    loss, acc = evaluate(params, data.train_ds)
    val_loss, val_acc = evaluate(params, data.val_ds)
    
    print('loss: {} - acc: {}'.format(loss, acc))
    print('val_loss: {} - val_acc: {}'.format(val_loss, val_acc))
    print('time: {}'.format(epoch_time))
    
    if args.wandb:
        wandb.log({"accuracy": acc, 
                   "val_accuracy": val_acc, 
                   'loss':loss, 
                   'val_loss':val_loss,
                   'lr': lr})


Validation: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 172.88batch/s]


loss: 0.34438323974609375 - acc: 0.5121951103210449
val_loss: 0.3683325946331024 - val_acc: 0.4000000059604645
time: 150.7751567363739


Validation: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 222.05batch/s]


loss: 0.35213714838027954 - acc: 0.5121951103210449
val_loss: 0.3668247163295746 - val_acc: 0.4000000059604645
time: 0.15316343307495117


Validation: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 232.07batch/s]


...

loss: 0.3332379460334778 - acc: 0.5243902206420898
val_loss: 0.3572011888027191 - val_acc: 0.4000000059604645
time: 0.15453290939331055


Validation: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 179.28batch/s]


loss: 0.3321480453014374 - acc: 0.5121951103210449
val_loss: 0.35221442580223083 - val_acc: 0.5
time: 0.17061781883239746


Validation: 100%|██████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 179.05batch/s]

loss: 0.33357712626457214 - acc: 0.4999999701976776
val_loss: 0.36186540126800537 - val_acc: 0.4000000059604645
time: 0.16035890579223633


In [147]:
from sklearn.metrics import roc_auc_score

out,y_train = predict(params, data.train_ds)
# _, y_test = tf_ds_to_numpy(data.test_ds)
train_auc = roc_auc_score(y_train, out)
train_auc

100%|███████████████████████████████████████████████████████████████████████████| 704/704 [03:58<00:00,  2.96batch/s]


0.7339170374074074

In [148]:
from sklearn.metrics import roc_auc_score

out,y_test = predict(params, data.test_ds)
# _, y_test = tf_ds_to_numpy(data.test_ds)
test_auc = roc_auc_score(y_test, out)
test_auc

100%|███████████████████████████████████████████████████████████████████████████| 157/157 [01:47<00:00,  1.47batch/s]


0.7229613925000001

In [149]:
if args.wandb:
    wandb.run.summary['test_loss'] = test_loss
    wandb.run.summary['test_acc'] = test_acc
    wandb.run.summary['test_auc'] = test_auc
    wandb.run.summary['train_auc'] = train_auc
    wandb.run.summary['avg_epoch_time'] = np.mean(np.array(epoch_times))
    y = y_test.argmax(axis=1)
    preds = out.argmax(axis=1)
    probs = out
    classes = data.mapping

    roc_curve = wandb.sklearn.plot_roc(y, probs, classes)
    confusion_matrix = wandb.sklearn.plot_confusion_matrix(y, preds, classes)

    wandb.log({"roc_curve": roc_curve})
    wandb.log({"confusion_matrix": confusion_matrix})

wandb: WARNING wandb uses only the first 10000 datapoints to create the plots.


In [150]:
if args.wandb:
    wandb.finish()

accuracy,▁▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇██████████████████
loss,█▆▅▄▄▄▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▁▁▁▁▁▁▁
lr,████████▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▂▃▄▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇███████████████████
val_loss,█▆▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.68082
avg_epoch_time,15.47968
loss,0.30514
lr,0.0
test_acc,0.66864
test_auc,0.72296


In [ ]:
for i in range(200):
    print(i, schedule_fn(i))